In [1]:
import sqlite3
import pandas as pd


In [2]:
#intialize the database from a script
DB_FILE = 'student_db'
initscript = 'initdb.sql'
with sqlite3.connect(DB_FILE) as con:
    with open(initscript,'r') as f:
        sql_commands = f.read()
        con.executescript(sql_commands)

In [3]:
# drop table 
con.execute('DROP TABLE IF EXISTS student')


In [4]:
con.execute('SELECT * FROM student')

OperationalError: no such table: student

In [5]:
# create the table again
with sqlite3.connect(DB_FILE) as con:
    con.execute('''CREATE TABLE IF NOT EXISTS student( 
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL, 
    age INT NOT NULL,
    id INT PRIMARY KEY
    );
    ''')
    con.commit() #confirm that the command has ran

In [6]:
# insert values
with sqlite3.connect(DB_FILE) as con:
    to_insert = [("Andrew","Rukangu",50,5),
                    ("Marie","Curie",100,6),
                    ("Spongebob", "Squarepants",50,7),
                    ("Tom", "Jerry",60,8)
                    ]
    con.executemany("INSERT INTO student VALUES (?,?,?,?)", to_insert)

In [8]:
with sqlite3.connect(DB_FILE) as con:
    data = con.execute('SELECT * FROM student').fetchall()
    print(data)

[('Andrew', 'Rukangu', 50, 5), ('Marie', 'Curie', 100, 6), ('Spongebob', 'Squarepants', 50, 7), ('Tom', 'Jerry', 60, 8)]


In [9]:
#insert one record
with sqlite3.connect(DB_FILE) as con:
    con.execute("INSERT INTO student VALUES(?,?,?,?)", ('Albert', 'Einstein',100,9))

In [10]:
with sqlite3.connect(DB_FILE) as con:      
    data = con.execute('SELECT * FROM student').fetchall()
    print(data)

[('Andrew', 'Rukangu', 50, 5), ('Marie', 'Curie', 100, 6), ('Spongebob', 'Squarepants', 50, 7), ('Tom', 'Jerry', 60, 8), ('Albert', 'Einstein', 100, 9)]


In [11]:
# select based on condition: age > 50
with sqlite3.connect(DB_FILE) as con:
    data=con.execute('SELECT * FROM student WHERE age>50').fetchall()
    print(data)

[('Marie', 'Curie', 100, 6), ('Tom', 'Jerry', 60, 8), ('Albert', 'Einstein', 100, 9)]


In [12]:
# read data into pandas df
with sqlite3.connect(DB_FILE) as con:
    df=pd.read_sql('SELECT * FROM student', con)
    df.head()
df

,first_name,last_name,age,id
0,Andrew,Rukangu,50,5
1,Marie,Curie,100,6
2,Spongebob,Squarepants,50,7
3,Tom,Jerry,60,8
4,Albert,Einstein,100,9


In [13]:
with sqlite3.connect(DB_FILE) as con:
    df2 = pd.read_sql("SELECT * FROM student ",con)
    display(df2)

,first_name,last_name,age,id
0,Andrew,Rukangu,50,5
1,Marie,Curie,100,6
2,Spongebob,Squarepants,50,7
3,Tom,Jerry,60,8
4,Albert,Einstein,100,9


In [14]:
with sqlite3.connect(DB_FILE) as con:
    df2 = pd.read_sql("SELECT * FROM student WHERE ROWID=1",con)
    display(df2)


,first_name,last_name,age,id
0,Andrew,Rukangu,50,5


In [15]:
with sqlite3.connect(DB_FILE) as con:
    df2 = pd.read_sql("SELECT * FROM student WHERE ROWID=2",con)
    display(df2)

,first_name,last_name,age,id
0,Marie,Curie,100,6


In [26]:
with sqlite3.connect(DB_FILE) as con:
    df2 = pd.read_sql("SELECT * FROM student WHERE ROWID=2",con)
    display(df2)

,first_name,last_name,age,id
0,Marie,Curie,100,6


In [27]:
with sqlite3.connect(DB_FILE) as con:
    command = "SELECT * FROM student WHERE ROWID=?"
    params = (2,)
    df2 = pd.read_sql(command,con,params=params)
    display(df2)

,first_name,last_name,age,id
0,Marie,Curie,100,6


In [21]:
#let's delete spongebob

with sqlite3.connect(DB_FILE) as con:
    print("Before deletion")
    df = pd.read_sql("SELECT * FROM student ",con)
    display(df)
    command = f"DELETE FROM student WHERE ROWID=?"
    params = (3,)
    con.execute(command,params,)
    print("After deletion")
    df2 = pd.read_sql("SELECT * FROM student ",con)
    display(df2)

Before deletion


,first_name,last_name,age,id
0,Andrew,Rukangu,50,5
1,Marie,Curie,100,6
2,Spongebob,Squarepants,50,7
3,Tom,Jerry,60,8
4,Albert,Einstein,100,9


After deletion


,first_name,last_name,age,id
0,Andrew,Rukangu,50,5
1,Marie,Curie,100,6
2,Tom,Jerry,60,8
3,Albert,Einstein,100,9


In [46]:
# rename table
command = '''ALTER TABLE student RENAME TO students;'''
con.execute(command)

In [16]:
items = df.to_records()
items

rec.array([(0, 'Andrew', 'Rukangu',  50, 5),
           (1, 'Marie', 'Curie', 100, 6),
           (2, 'Spongebob', 'Squarepants',  50, 7),
           (3, 'Tom', 'Jerry',  60, 8), (4, 'Albert', 'Einstein', 100, 9)],
          dtype=[('index', '<i8'), ('first_name', 'O'), ('last_name', 'O'), ('age', '<i8'), ('id', '<i8')])

In [17]:
for item in items:
    print(item.index, item.first_name)

0 Andrew
1 Marie
2 Spongebob
3 Tom
4 Albert


In [ ]:
# select specific columns 
command = '''SELECT first_name, last_name FROM students'''
con.execute(command).fetchall()

In [45]:
# search using LIKE

command = '''SELECT * FROM students WHERE first_name LIKE "a%"'''
con.execute(command).fetchall()

OperationalError: no such table: students

- %	Represents zero or more characters
- _	Represents a single character
- []	Represents any single character within the brackets *
- ^	Represents any character not in the brackets *
- \-	Represents any single character within the specified range *
- {}	Represents any escaped character **

In [ ]:
# insert one record using dict method
command = """INSERT INTO students (first_name, last_name,age,id) VALUES(:fname,:lname,:age,:id)"""
params = {'fname':'Von', 'lname':'Neumann','age':60,'id':12}
con.execute(command,params)
con.execute('SELECT * FROM students').fetchall()

In [ ]:
with sqlite3.connect(DB_FILE) as con:
    df=pd.read_sql('SELECT * FROM students', con)
 

In [ ]:
df.head()

In [120]:
def create_sql(columns):
    sql_statement = ''
    cols = ''
    for column in columns:
        sql_statement+= f'''{column[1]} {column[2]} {'NOT NULL' if column[3]==1 else ''} {'DEFAULT ' + str(column[4]) if column[4] else '' } {'PRIMARY KEY' if column[4] else ''},'''
        cols+=f'{column[1]},'
    return cols.strip(','),sql_statement.strip(',')

In [28]:
def check_and_add_id_column(tablename, DB_FILE):
    # Connect to the database
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    # Check if 'id' column exists
    cursor.execute(f"PRAGMA table_info({tablename});")
    columns = cursor.fetchall()
    id_column_info = next((column for column in columns if column[1] == 'id'), None)

    if id_column_info:
        # 'id' column exists
        id_column_index = id_column_info[0]
        is_primary_key = id_column_info[5] == 1

        if not is_primary_key:
            # 'id' column exists but is not a primary key, change it to a primary key
            cursor.execute(f"CREATE TEMPORARY TABLE temp_table AS SELECT * FROM {tablename};")
            cursor.execute(f"DROP TABLE {tablename};")
            cols,commands = create_sql(columns)
            cursor.execute(f"CREATE TABLE {tablename} (id INTEGER PRIMARY KEY, {commands} );")
            cursor.execute(f"INSERT INTO {tablename} ({cols})SELECT {cols} FROM temp_table;")
            cursor.execute("DROP TABLE temp_table;")
            conn.commit()

    else:
        # 'id' column does not exist, add it
        cursor.execute("ALTER TABLE {tablename} ADD COLUMN id INTEGER PRIMARY KEY;")
        cursor.execute("UPDATE {tablename} SET id = rowid;")

    # Commit changes and close connection
    conn.commit()
    conn.close()

In [31]:
conn = sqlite3.connect(DB_FILE)

In [32]:
cursor = conn.cursor()

# Check if 'id' column exists
cursor.execute(f"PRAGMA table_info(student);")
columns = cursor.fetchall()


In [33]:
columns

[(0, 'first_name', 'TEXT', 1, None, 0),
 (1, 'last_name', 'TEXT', 1, None, 0),
 (2, 'age', 'INT', 1, None, 0),
 (3, 'id', 'INT', 0, None, 1)]

In [40]:
id_column_info = next((column for column in columns if column[1] == 'id'), None)
id_column_info

(3, 'id', 'INT', 0, None, 1)

In [41]:
id_column_info[5]

1

In [102]:
con.close()

In [111]:
conn.close()

In [112]:
conn = sqlite3.connect('inventory_db.db')

In [113]:
cursor = conn.cursor()
# Check if 'id' column exists
cursor.execute(f"PRAGMA table_info(inventory);")
columns = cursor.fetchall()
columns

[(0, 'category', 'TEXT', 1, None, 0),
 (1, 'product_description', 'TEXT', 1, None, 0),
 (2, 'price', 'FLOAT', 1, None, 0),
 (3, 'code', 'TEXT', 1, None, 0)]

In [114]:
id_column_info = next((column for column in columns if column[1] == 'id'), None)
id_column_info

In [115]:
def create_sql(columns):
    sql_statement = ''
    cols = ''
    for column in columns:
        sql_statement+= f'''{column[1]} {column[2]} {'NOT NULL' if column[3]==1 else ''} {'DEFAULT ' + str(column[4]) if column[4] else '' } {'PRIMARY KEY' if column[4] else ''},'''
        cols+=f'{column[1]},'
    return cols.strip(','),sql_statement.strip(',')

In [118]:
tablename = 'inventory'
cursor.execute(f"CREATE TEMPORARY TABLE temp_table AS SELECT * FROM {tablename};")
cursor.execute(f"DROP TABLE {tablename};")
cols,commands = create_sql(columns)
cursor.execute(f"CREATE TABLE {tablename} (id INTEGER PRIMARY KEY, {commands} );")
cursor.execute(f"INSERT INTO {tablename} ({cols})SELECT {cols} FROM temp_table;")
cursor.execute("DROP TABLE temp_table;")
conn.commit()

In [119]:
df = pd.read_sql("SELECT * FROM inventory ",conn)
display(df)

,id,category,product_description,price,code
0,1,Electronics,ieee,54.0,B01
1,2,Books,The Art of Electronics,213.0,B01
2,3,Boks,The Grumpy Mountain Man,2222222.0,b00


In [98]:
cols, commands = create_sql(columns)
print(f"CREATE TABLE {tablename} (id INTEGER PRIMARY KEY,{commands} );")
print(cols)

CREATE TABLE inventory (id INTEGER PRIMARY KEY,category TEXT NOT NULL  ,product_description TEXT NOT NULL  ,price FLOAT NOT NULL  ,code TEXT NOT NULL   );
category,product_description,price,code


In [93]:
commands

'category TEXT NOT NULL  ,product_description TEXT NOT NULL  ,price FLOAT NOT NULL  ,code TEXT NOT NULL  ,'

In [94]:
cols

[(0, 'category', 'TEXT', 1, None, 0),
 (1, 'product_description', 'TEXT', 1, None, 0),
 (2, 'price', 'FLOAT', 1, None, 0),
 (3, 'code', 'TEXT', 1, None, 0)]

In [86]:
columns

[(0, 'category', 'TEXT', 1, None, 0),
 (1, 'product_description', 'TEXT', 1, None, 0),
 (2, 'price', 'FLOAT', 1, None, 0),
 (3, 'code', 'TEXT', 1, None, 0)]

In [54]:
f"{columns[0][1]} {columns[0][2]} {'NOT NULL' if columns[0][3]==1 else ''}"

'category TEXT NOT NULL'

In [57]:
create_sql(columns)

'category TEXT NOT NULL  ;product_description TEXT NOT NULL  ;price FLOAT NOT NULL  ;code TEXT NOT NULL  ;'

In [29]:
with sqlite3.connect(DB_FILE) as con:
    df = pd.read_sql("SELECT * FROM student ",con)
    display(df)

,first_name,last_name,age,id
0,Andrew,Rukangu,50,5
1,Marie,Curie,100,6
2,Tom,Jerry,60,8
3,Albert,Einstein,100,9


In [30]:
check_and_add_id_column()
with sqlite3.connect(DB_FILE) as con:
    df = pd.read_sql("SELECT * FROM student ",con)
    display(df)

,first_name,last_name,age,id
0,Andrew,Rukangu,50,5
1,Marie,Curie,100,6
2,Tom,Jerry,60,8
3,Albert,Einstein,100,9
